# Servidor con generador de voz sintética y traducción en ambas direcciones usando zephyr model.

Ten en cuenta activar CORS para que no de error:
%pip install flask-cors

In [45]:
%%writefile server_conversacional.py
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import MarianMTModel, MarianTokenizer
from threading import Lock
import torch

import time
# Asegúrate de importar tu modelo Whisper correctamente
# from tu_paquete import modelWhisper

import whisper
modelWhisper = whisper.load_model('medium')


model_name = 'Helsinki-NLP/opus-mt-es-en'  # Modelo para traducir de español a inglés
modelo_traductor = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)


def translate_text_to_english(text):
    print("Traduciendo texto:", text)
    tokens = tokenizer(text, return_tensors='pt', padding=True)
    translated = modelo_traductor.generate(**tokens)
    decoded = []
    for t in translated:
        decoded.append(tokenizer.decode(t, skip_special_tokens=True))

    return decoded[0]


model_name = 'Helsinki-NLP/opus-mt-tc-big-en-es'  # Modelo para traducir de inglés a español
model_big_en_es = MarianMTModel.from_pretrained(model_name)
tokenizer_big_en_es = MarianTokenizer.from_pretrained(model_name)

def translate_text_to_spanish(text):
      tokenizer = tokenizer_big_en_es
      model = model_big_en_es

      tokens = tokenizer(text, return_tensors='pt', padding=True)
      translated = model.generate(**tokens)
      # decoded = tokenizer.decode(translation[0], skip_special_tokens=True)
      decoded = []
      for t in translated:
          # decoded = tokenizer.decode(t, skip_special_tokens=True)
          decoded.append(tokenizer.decode(t, skip_special_tokens=True))
      
      return decoded[0]


modelo = "zypher"

if modelo == "mistral":
    from modelo_mistral_base import generate_long_chat, load_model
elif modelo == "zypher":
    from modelo_Zypher_beta import generate_long_chat, load_model
else:
    print("modelo no encontrado")
    exit()

ai = "assistant"
user = "user"

contexto = """

"""

system_prompt = """
You are a kind and helpful assistan bot. You are here to help the user to find the best answer to his question.
"""

saludo = "Hello, I am ready to receive and process your input."

idioma = "en"

import sys

# Verifica si el comando tenía flag -s o --short
if "-s" in sys.argv or "--short" in sys.argv:
    short_answer = True

# Si encuentra el flag -es cambia el idioma a español
if "-es" in sys.argv:
    idioma = "es"

# Filtra los argumentos para eliminar los flags
args = [arg for arg in sys.argv[1:] if arg not in ["-s", "--short", "-es"]]

# Asigna los valores a system_prompt y saludo basándose en los argumentos restantes
if len(args) > 0:
    system_prompt = args[0]
if len(args) > 1:
    saludo = args[1]

if modelo == "mistral":
    historico = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>assistant\n{saludo}<|im_end|>\n"
elif modelo == "zypher":
    historico = f"<|system|>{system_prompt}</s>\n<|assistant|>\n{saludo}</s>\n"


# load model
load_model(user=user, ai=ai)

print(f"{ai}:", saludo)

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de transcribir
transcribe_lock = Lock()

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de traducir
translate_lock = Lock()

# Crea un bloqueo para proteger el código contra la concurrencia a la hora de generar texto
generate_lock = Lock()




app = Flask(__name__)
CORS(app)


@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    global historico
    global user
    global ai
    # global iteracion

    traduccion = ""

    # Comprueba si el archivo fue enviado
    if 'file' not in request.files:
        return jsonify(error="No file part"), 400

    file = request.files['file']

    # Comprueba si el usuario no seleccionó un archivo
    if file.filename == '':
        return jsonify(error="No selected file"), 400

    # Genera un nombre de archivo único utilizando una marca de tiempo
    timestamp = int(time.time() * 1000)  # Marca de tiempo en milisegundos
    mp3_filepath = f"received_audio_{timestamp}.mp3"
    file.save(mp3_filepath)

    # Transcribe el archivo MP3 (Asegúrate de tener el modelo cargado correctamente)
    # Transcribe el archivo MP3 dentro de una sección crítica protegida por un bloqueo
    with transcribe_lock:
        # transcripcion = modelWhisper.transcribe(mp3_filepath, fp16=False)
        # transcipción lenguaje inglés
        transcripcion = modelWhisper.transcribe(mp3_filepath, fp16=False, language=idioma)
        transcripcion = transcripcion["text"]
        print("transcripción:", transcripcion)

    # si el idioma es español, traduce la transcripción al inglés
    if idioma == "es":
        with translate_lock:
            traduccion = translate_text_to_english(transcripcion)
        print("traducción:", traduccion)
        entrada = traduccion
    else:
        entrada = transcripcion
    # prompt = f"{historico}\n{user}:{entrada}\n{ai}:"
    # print("prompt:", prompt)


    with generate_lock:
        historico, output = generate_long_chat(historico, ai, user, input_text=entrada, max_additional_tokens=2048, short_answer=short_answer, streaming=False, printing=False)
        print("output:", output)
        print("historico:", historico)
      
    # si el idioma es español, traduce la respuesta al español
    if idioma == "es":
        with translate_lock:
            traduccion_res = translate_text_to_spanish(output)
        print("traducción:", traduccion_res)
        respuesta_trad = traduccion_res



    return jsonify(entrada=transcripcion, entrada_traducida=traduccion, respuesta=output, respuesta_traducida=respuesta_trad)



@app.route('/texto', methods=['POST'])
def process_text():
    global historico
    global user
    global ai

    # Recibe el texto directamente del cuerpo de la solicitud
    data = request.json
    if not data or 'texto' not in data:
        return jsonify(error="No se proporcionó texto"), 400

    texto = data['texto']
    
    # Utiliza la variable 'idioma' declarada globalmente
    global idioma

    # Si el idioma es español, traduce la transcripción al inglés
    traduccion = ""
    if idioma == "es":
        with translate_lock:
            traduccion = translate_text_to_english(texto)
        entrada = traduccion
    else:
        entrada = texto

    # Generación de respuesta basada en el texto proporcionado
    with generate_lock:
        historico, output = generate_long_chat(historico, ai, user, input_text=entrada, max_additional_tokens=2048, short_answer=short_answer, streaming=False, printing=False)

    # si el idioma es español, traduce la respuesta al español
    if idioma == "es":
        with translate_lock:
            traduccion_res = translate_text_to_spanish(output)
        print("traducción:", traduccion_res)
        respuesta_trad = traduccion_res



    return jsonify(entrada=texto, entrada_traducida=traduccion, respuesta=output, respuesta_traducida=respuesta_trad)



# PREPARAMOS INSTANCIAS DE OpenVoice

if idioma == "en":
    import os
    openvoice_dir = os.path.abspath("OpenVoice")
    import sys
    sys.path.append(openvoice_dir)

    import torch
    import se_extractor
    from api import BaseSpeakerTTS, ToneColorConverter

    open_voice_dir = '/content/OpenVoice'
    output_dir = f"{open_voice_dir}/output"
    save_path = f'{output_dir}/output_en_default.wav'

    ckpt_base = f"{open_voice_dir}/checkpoints/base_speakers/EN"
    ckpt_converter = f"{open_voice_dir}/checkpoints/converter"
    device="cuda:0" if torch.cuda.is_available() else "cpu"

    base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
    base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

    tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
    tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

    os.makedirs(output_dir, exist_ok=True)


    source_se = torch.load(f'{ckpt_base}/en_style_se.pth').to(device)
    # source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

    reference_speaker = f"{open_voice_dir}/resources/demo_speaker0.mp3"
    # reference_speaker = "resources/javier.mp3"
    target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)


    save_path = f'{output_dir}/output_whispering.wav'



elif idioma == "es":
    # El modelo no entiende de números aritméticos. Esta función los convierte a palabras.
    import re
    from num2words import num2words

    def number_to_words(num_str):
        try:
            num = int(num_str)
            return num2words(num, lang='es')
        except ValueError:
            return "Por favor, introduzca un número válido."

    def process_numbers_in_line(line):
        def replace_with_words(match):
            return number_to_words(match.group())
        
        return re.sub(r'\b\d+\b', replace_with_words, line)

    # Ejemplo de uso
    line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
    new_line = process_numbers_in_line(line)
    print(new_line)  
    # Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."


    # Diccionario con las traducciones

    def process_abrev(line):
        translations = {
        'Dr': 'doctor',
        'Sr': 'señor',
        'Sra': 'señora',
        # Añade más traducciones aquí
    }
        for abbr, full in translations.items():
            line = line.replace(f'{abbr}.', full)
            line = line.replace(f'{abbr} ', f'{full} ')
        return line

    def otras_traducciones(line):

        translations = {
        '-': ',',
        '—': ',', 
        '%': ' por ciento '
        # Añade más traducciones aquí
        }
        
        for old, new in translations.items():
            line = line.replace(old, new)
        return line        


    def preprocesado_al_modelo(line):
        line_with_numbers = process_numbers_in_line(line)
        line_with_both = process_abrev(line_with_numbers)
        line_with_all = otras_traducciones(line_with_both)
        return line_with_all

    import os
    os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
    print(os.environ['TF_ENABLE_ONEDNN_OPTS'])


    # Cargamos el modelo generador fairseq
    from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
    from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
    import IPython.display as ipd
    import torch  # Importamos PyTorch para poder usar la función `to()`

    # Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
    def move_to_device(obj, device):
        if isinstance(obj, torch.Tensor):
            return obj.to(device)
        elif isinstance(obj, dict):
            return {key: move_to_device(value, device) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [move_to_device(item, device) for item in obj]
        else:
            return obj
    

    # Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
    models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
        "facebook/tts_transformer-es-css10",
        arg_overrides={"vocoder": "hifigan", "fp16": False}
    )
    model = models[0]

    # Movemos el modelo al dispositivo GPU
    model = model.to('cuda:0')

    # Actualizamos la configuración con los datos del task
    TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

    # Creamos el generador
    generator = task.build_generator([model], cfg)

    
    import torchaudio

    import re

    def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
        partes = re.split(r'([.;:?!])', texto)
        partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
        partes_combinadas = []
        parte_actual = ''

        for parte in partes_filtradas:
            if parte in '.;:?!':
                parte_actual += parte
                if len(parte_actual.split()) >= min_palabras:
                    partes_combinadas.append(parte_actual)
                    parte_actual = ''
                else:
                    parte_actual += ' '
            else:
                parte_actual += parte + ' '

        if parte_actual.strip():
            partes_combinadas.append(parte_actual.strip())

        return partes_combinadas

    def combinar_audios(audios_temporales):
        # Cargar el primer archivo de audio para inicializar la concatenación
        wav_total, rate = torchaudio.load(audios_temporales[0])

        # Iterar sobre los archivos restantes y concatenarlos
        for archivo in audios_temporales[1:]:
            wav, _ = torchaudio.load(archivo)
            wav_total = torch.cat((wav_total, wav), 1)

        # Guardar el audio combinado en un archivo final
        torchaudio.save('audio_combinado.wav', wav_total, rate)

        return 'audio_combinado.wav'

    def voz_sintetica_espanish(text):
        text = preprocesado_al_modelo(text)

        lista_dividida = dividir_texto_con_minimo_palabras(text)

        audios_temporales = []
        
        for parte in lista_dividida:
            # Preparamos los datos de entrada para el modelo
            sample = TTSHubInterface.get_model_input(task, parte)

            # Movemos los datos al dispositivo GPU
            sample = move_to_device(sample, 'cuda:0')

            # Realizamos la predicción
            wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

            if len(wav.shape) == 1:
                wav = wav.unsqueeze(0)
                
            # temp_file_name = "Temporal.wav"
            temp_file_name = f"temporal_{parte[:10]}.wav"
            torchaudio.save(temp_file_name, wav.to('cpu'), rate)
            audios_temporales.append(temp_file_name)

            archivo_final = combinar_audios(audios_temporales)


        with open(archivo_final, "rb") as audio_file:
            audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

        return audio_base64



import base64
@app.route('/audio', methods=['POST'])
def generate_audio():
    texto = request.json.get('texto', '')

    if not texto:
        return jsonify(error="No se proporcionó texto"), 400

    if idioma == "en":
        audio_base64 = voz_sintetica_english(texto)
        return jsonify(audio_base64=audio_base64)
    elif idioma == "es":
        audio_base64 = voz_sintetica_espanish(texto)
        return jsonify(audio_base64=audio_base64)

def voz_sintetica_english(texto):
    src_path = 'tmp.wav'

    base_speaker_tts.tts(texto, src_path, speaker='default', language='English', speed=0.9)
  # Run the tone color converter
    encode_message = "@MyShell"
    tone_color_converter.convert(
        audio_src_path=src_path,
        src_se=source_se,
        tgt_se=target_se,
        output_path=save_path,
        message=encode_message)

    with open(save_path, "rb") as audio_file:
        audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

    return audio_base64



if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5500, threaded=True)


Overwriting server_conversacional.py


## pm2 ejecutor de procesos en segundo plano

In [9]:
%%writefile server_conversacional.config.js
module.exports = {
  apps: [
    {
      name: "server_conversacional",
      script: "server_conversacional.py",
      args: ["--short", "-es", "You are Edward, the CEO of a consulting company called mAgIc, dedicated to finding artificial intelligence solutions for companies. You are interviewing a candidate to work for the company, you are interested in programming skills, mathematics, data science, AI and especially NLP. Ask questions about all this and probe. If the candidate says something wrong let them know. After 4 questions if most were good responses you must say 'CONGRATULATION YOU ARE IN OUR TEAM!!' if there was a bad response then you must say 'The interview is finished, thanks for come' IMPORTANT: if the interviewee is rude then you must say 'go away, close the door when go out'", "Good morning, wellcome to MAgIc, have a seat, tell me your name."],
      out_file: "out.log",
      error_file: "err.log",
      log_file: "combined.log",
      time: true,
    },
  ],
};


Overwriting server_conversacional.config.js


In [47]:
!pm2 start server_conversacional.config.js

[PM2][WARN] Applications server_conversacional not running, starting...
[PM2] App [server_conversacional] launched (1 instances)
┌────┬────────────────────┬──────────┬──────┬───────────┬──────────┬──────────┐
│ id │ name               │ mode     │ ↺    │ status    │ cpu      │ memory   │
├────┼────────────────────┼──────────┼──────┼───────────┼──────────┼──────────┤
│ 0  │ server_conversaci… │ fork     │ 0    │ online    │ 0%       │ 5.3mb    │
└────┴────────────────────┴──────────┴──────┴───────────┴──────────┴──────────┘


In [8]:
!pm2 list

┌────┬────────────────────┬──────────┬──────┬───────────┬──────────┬──────────┐
│ id │ name               │ mode     │ ↺    │ status    │ cpu      │ memory   │
└────┴────────────────────┴──────────┴──────┴───────────┴──────────┴──────────┘


In [46]:
%%bash
pm2 stop 0
pm2 delete 0

[PM2] Applying action stopProcessId on app [0](ids: [ '0' ])
[PM2] [server_conversacional](0) ✓
┌────┬──────────────────────────┬─────────────┬─────────┬─────────┬──────────┬────────┬──────┬───────────┬──────────┬──────────┬──────────┬──────────┐
│ id │ name                     │ namespace   │ version │ mode    │ pid      │ uptime │ ↺    │ status    │ cpu      │ mem      │ user     │ watching │
├────┼──────────────────────────┼─────────────┼─────────┼─────────┼──────────┼────────┼──────┼───────────┼──────────┼──────────┼──────────┼──────────┤
│ 0  │ server_conversacional    │ default     │ N/A     │ fork    │ 0        │ 0      │ 0    │ stopped   │ 0%       │ 0b       │ javier   │ disabled │
└────┴──────────────────────────┴─────────────┴─────────┴─────────┴──────────┴────────┴──────┴───────────┴──────────┴──────────┴──────────┴──────────┘
[PM2] Applying action deleteProcessId on app [0](ids: [ '0' ])
[PM2] [server_conversacional](0) ✓
┌────┬───────────┬─────────────┬─────────┬─────────

## Cliente javascript

In [21]:
%%html

<div id="audioPlayerContainer"></div>
<button id="recordButton" style="width: 100%; height: 50px;">Pulsa para grabar/detener</button>

<!-- Estilos para textarea y botón de envío -->
<div style="margin-top: 10px;">
    <textarea id="textInput" placeholder="Escribe tu texto aquí" rows="4" style="width: calc(100% - 90px); display: inline-block;"></textarea>
    <button id="sendTextButton" style="width: 80px; height: 70px; vertical-align: top;">Enviar Texto</button>
</div>


<div id="responseText" style="margin-top: 20px; border: 1px solid #ddd; padding: 10px;"></div>
<script>
let recordButton = document.getElementById("recordButton");
let chunks = [];
let mediaRecorder;
let isRecording = false;

navigator.mediaDevices.getUserMedia({ audio: true })
.then(stream => {
    mediaRecorder = new MediaRecorder(stream);
    mediaRecorder.ondataavailable = event => {
        chunks.push(event.data);
    };
    mediaRecorder.onstop = () => {
        let blob = new Blob(chunks, { 'type': 'audio/ogg; codecs=opus' });
        enviarAudioAlServidor(blob);
        chunks = [];
    };
});

recordButton.onclick = () => {
    if (!isRecording) {
        mediaRecorder.start();
        isRecording = true;
        recordButton.textContent = 'Grabando...';
    } else {
        mediaRecorder.stop();
        isRecording = false;
        recordButton.textContent = 'Pulsa para grabar/detener';
    }
};

function enviarAudioAlServidor(blob) {
    let formData = new FormData();
    formData.append('file', blob, 'grabacion.ogg');

    fetch('http://localhost:5500/transcribe', {
        method: 'POST',
        body: formData,
    })
    .then(response => response.json())
    .then(data => {        
        updateResponseText(data.entrada + "\n" + (data.entrada_traducida ? data.entrada_traducida + "\n" : "") + data.respuesta + "\n" + (data.respuesta_traducida ? data.respuesta_traducida + "\n" : ""));
        if (data.respuesta_traducida) {
            obtenerYReproducirAudio(data.respuesta_traducida);
        }
        else if (data.respuesta) {
            obtenerYReproducirAudio(data.respuesta);
        }
    })
    .catch(error => {
        console.error('Error al enviar el audio:', error);
    });
}

// Funcionalidad para enviar texto al servidor y limpiar el textarea
document.getElementById("sendTextButton").onclick = () => {
    let textInput = document.getElementById("textInput");
    let texto = textInput.value;
    if (texto) {
        enviarTextoAlServidor(texto);
        textInput.value = ''; // Limpiar el textarea después de enviar
    }
};

function enviarTextoAlServidor(texto) {
    fetch('http://localhost:5500/texto', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto })
    })
    .then(response => response.json())
    .then(data => {
        updateResponseText(data.entrada + "\n" + (data.entrada_traducida ? data.entrada_traducida + "\n" : "") + data.respuesta + "\n" + (data.respuesta_traducida ? data.respuesta_traducida + "\n" : ""));
        if (data.respuesta_traducida) {
            obtenerYReproducirAudio(data.respuesta_traducida);
        }
        else if (data.respuesta) {
            obtenerYReproducirAudio(data.respuesta);
        }
    })
    .catch(error => {
        console.error('Error al enviar el texto:', error);
    });
}

function updateResponseText(text) {
    document.getElementById('responseText').innerText = text;
}

function obtenerYReproducirAudio(texto) {
    fetch('http://localhost:5500/audio', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ texto: texto })
    })
    .then(response => response.json())
    .then(data => {
        if (data.audio_base64) {
            createAudioPlayer(data.audio_base64);
        }
    })
    .catch(error => {
        console.error('Error al obtener el audio:', error);
    });
}

function createAudioPlayer(base64Audio) {
    let audioContainer = document.getElementById('audioPlayerContainer');
    let audioSrc = `data:audio/wav;base64,${base64Audio}`;
    let audioPlayer = document.createElement('audio');
    audioPlayer.src = audioSrc;
    audioPlayer.controls = true;
    audioPlayer.autoplay = true;
    audioContainer.innerHTML = '';
    audioContainer.appendChild(audioPlayer);
}
</script>

# Comprobaciones

In [22]:
!lsof -i :5500

COMMAND  PID   USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python  3510 javier   20u  IPv4  48287      0t0  TCP *:5500 (LISTEN)


In [23]:
!nc -vz localhost 5500

Connection to localhost (127.0.0.1) 5500 port [tcp/*] succeeded!


# Verificamos funcionamiento de generador de voz sintética.

In [13]:
import re
from num2words import num2words

def number_to_words(num_str):
    try:
        num = int(num_str)
        return num2words(num, lang='es')
    except ValueError:
        return "Por favor, introduzca un número válido."

def process_numbers_in_line(line):
    def replace_with_words(match):
        return number_to_words(match.group())
    
    return re.sub(r'\b\d+\b', replace_with_words, line)

# Ejemplo de uso
line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
new_line = process_numbers_in_line(line)
print(new_line)  
# Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."

Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas.


In [15]:
# Diccionario con las traducciones


def process_abrev(line):
    translations = {
    'Dr': 'doctor',
    'Sr': 'señor',
    'Sra': 'señora',
    # Añade más traducciones aquí
}
    for abbr, full in translations.items():
        line = line.replace(f'{abbr}.', full)
        line = line.replace(f'{abbr} ', f'{full} ')
    return line

# Ejemplo de uso
line1 = 'El Dr. está hablando con el Sr. Pérez y la Sra. Gómez.'
line2 = 'Buenos días Sr. Gómez.'

new_line1 = process_abrev(line1)
new_line2 = process_abrev(line2)

print(new_line1)  # Debería imprimir: 'El Doctor está hablando con el Señor Pérez y la Señora Gómez.'
print(new_line2)  # Debería imprimir: 'Buenos días Señor Gómez.'

El doctor está hablando con el señor Pérez y la señora Gómez.
Buenos días señor Gómez.


In [16]:
# Diccionario con las traducciones


def otras_traducciones(line):

    translations = {
    '-': ',',
    '—': ',', 
    '%': ' por ciento '
    # Añade más traducciones aquí
    }
    
    for old, new in translations.items():
        line = line.replace(old, new)
    return line

# Ejemplo de uso
line1 = 'Hola —cómo estás—?, bien al 100%, gracias.'
line2 = "El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón —el primer hobbit que fue famoso en el mundo entero— y que él tituló Historia de una ida y de una vuelta,"

new_line1 = otras_traducciones(line1)
new_line2 = otras_traducciones(line2)

print(new_line1)  # Debería imprimir: 'Hola, cómo estás?'
print(new_line2)  # Debería imprimir: 'Bien, gracias.'

Hola ,cómo estás,?, bien al 100 por ciento , gracias.
El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón ,el primer hobbit que fue famoso en el mundo entero, y que él tituló Historia de una ida y de una vuelta,


In [17]:
def preprocesado_al_modelo(line):
    line_with_numbers = process_numbers_in_line(line)
    line_with_both = process_abrev(line_with_numbers)
    line_with_all = otras_traducciones(line_with_both)
    return line_with_all

In [18]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
print(os.environ['TF_ENABLE_ONEDNN_OPTS'])  # Salida: mi_valor

0


In [19]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd
import torch  # Importamos PyTorch para poder usar la función `to()`

# Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
def move_to_device(obj, device):
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, device) for item in obj]
    else:
        return obj
    

# Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-es-css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
model = models[0]

# Movemos el modelo al dispositivo GPU
model = model.to('cuda:0')

# Actualizamos la configuración con los datos del task
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

# Creamos el generador
generator = task.build_generator([model], cfg)


# Texto a convertir en voz
text = "Hola, esta es una prueba. Quiero saber hasta dónde llega el modelo."

# Preparamos los datos de entrada para el modelo
sample = TTSHubInterface.get_model_input(task, text)

# Movemos los datos al dispositivo GPU
sample = move_to_device(sample, 'cuda:0')

# Realizamos la predicción
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

# Convertimos el resultado al formato de audio y lo reproducimos
ipd.Audio(wav.cpu(), rate=rate)  # Usamos `.cpu()` para mover los datos de vuelta a la CPU antes de reproducir el audio


2024-01-31 23:59:18 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

2024-01-31 23:59:20 | INFO | fairseq.tasks.speech_to_text | dictionary size (spm_char.txt): 107
/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
2024-01-31 23:59:20 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/javier/.cache/fairseq/models--facebook--tts_transformer-es-css10/snapshots/f52cf36f741df546bed60cdd5e6b71e0b85378c1/hifigan.bin
2024-01-31 23:59:20 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/javier/.cache/fairseq/models--facebook--tts_transformer-es-css10/snapshots/f52cf36f741df546bed60cdd5e6b71e0b85378c1/hifigan.bin
/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning:

In [40]:
import re

def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
    partes = re.split(r'([.;:?!])', texto)
    partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
    partes_combinadas = []
    parte_actual = ''

    for parte in partes_filtradas:
        if parte in '.;:?!':
            parte_actual += parte
            if len(parte_actual.split()) >= min_palabras:
                partes_combinadas.append(parte_actual)
                parte_actual = ''
            else:
                parte_actual += ' '
        else:
            parte_actual += parte + ' '

    if parte_actual.strip():
        partes_combinadas.append(parte_actual.strip())

    return partes_combinadas

texto = "Genial, gracias por venir hoy. Como sabes, estamos buscando a alguien con fuertes habilidades de programación, matemáticas, ciencia de datos, inteligencia artificial y experiencia en PNL para unirse a nuestro equipo. Comencemos con los lenguajes de programación, ¿cuáles eres más competente?"
lista_dividida = dividir_texto_con_minimo_palabras(texto)

print(lista_dividida)


['Genial, gracias por venir hoy . Como sabes, estamos buscando a alguien con fuertes habilidades de programación, matemáticas, ciencia de datos, inteligencia artificial y experiencia en PNL para unirse a nuestro equipo .', 'Comencemos con los lenguajes de programación, ¿cuáles eres más competente ?']


## Prueba con traducción larga

In [39]:
# Texto a convertir en voz
text = "Genial, gracias por venir hoy. Como sabes, estamos buscando a alguien con fuertes habilidades de programación, matemáticas, ciencia de datos, inteligencia artificial y experiencia en PNL para unirse a nuestro equipo. Comencemos con los lenguajes de programación, ¿cuáles eres más competente?"

lista_textos = dividir_texto_con_minimo_palabras(text, min_palabras=7)

for texto in lista_textos:
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, texto)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

    # Convertimos el resultado al formato de audio y lo reproducimos
    display(ipd.Audio(wav.cpu(), rate=rate))  # Usamos `.cpu()` para mover los datos de vuelta a la CPU antes de reproducir el audio


In [44]:
import torchaudio

import re

def dividir_texto_con_minimo_palabras(texto, min_palabras=8):
    partes = re.split(r'([.;:?!])', texto)
    partes_filtradas = [parte.strip() for parte in partes if parte.strip()]
    partes_combinadas = []
    parte_actual = ''

    for parte in partes_filtradas:
        if parte in '.;:?!':
            parte_actual += parte
            if len(parte_actual.split()) >= min_palabras:
                partes_combinadas.append(parte_actual)
                parte_actual = ''
            else:
                parte_actual += ' '
        else:
            parte_actual += parte + ' '

    if parte_actual.strip():
        partes_combinadas.append(parte_actual.strip())

    return partes_combinadas

def combinar_audios(audios_temporales):
    # Cargar el primer archivo de audio para inicializar la concatenación
    wav_total, rate = torchaudio.load(audios_temporales[0])

    # Iterar sobre los archivos restantes y concatenarlos
    for archivo in audios_temporales[1:]:
        wav, _ = torchaudio.load(archivo)
        wav_total = torch.cat((wav_total, wav), 1)

    # Guardar el audio combinado en un archivo final
    torchaudio.save('temporal_combinado.wav', wav_total, rate)

    return 'temporal_combinado.wav'

def voz_sintetica_espanish(text):
    text = preprocesado_al_modelo(text)

    lista_dividida = dividir_texto_con_minimo_palabras(text)

    audios_temporales = []
    
    for parte in lista_dividida:
        # Preparamos los datos de entrada para el modelo
        sample = TTSHubInterface.get_model_input(task, parte)

        # Movemos los datos al dispositivo GPU
        sample = move_to_device(sample, 'cuda:0')

        # Realizamos la predicción
        wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

        if len(wav.shape) == 1:
            wav = wav.unsqueeze(0)
            
        # temp_file_name = "Temporal.wav"
        temp_file_name = f"temporal_{parte[:10]}.wav"
        torchaudio.save(temp_file_name, wav.to('cpu'), rate)
        audios_temporales.append(temp_file_name)

        archivo_final = combinar_audios(audios_temporales)

text = "Genial, gracias por venir hoy. Como sabes, estamos buscando a alguien con fuertes habilidades de programación, matemáticas, ciencia de datos, inteligencia artificial y experiencia en PNL para unirse a nuestro equipo. Comencemos con los lenguajes de programación, ¿cuáles eres más competente?"

voz_sintetica_espanish(text)

In [ ]:
def spanish_t2s(text):
    text = preprocesado_al_modelo(text)
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, text)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

    return wav, rate